<a href="https://colab.research.google.com/github/soumyadip-Roy/BLE_Module/blob/main/ble_collab_build_apk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install kivy pyjnius buildozer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 36.6 MB/s eta 0:00:00


In [10]:
!buildozer init

Buildozer is running as root!
This is not recommended, and may lead to problems later.
Are you sure you want to continue [y/n]? y
File buildozer.spec created, ready to customize!


In [7]:
# main.py
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.button import Button
from kivy.uix.label import Label
from kivy.uix.textinput import TextInput
from kivy.uix.recycleview import RecycleView
from jnius import autoclass

# Android classes
BluetoothAdapter = autoclass('android.bluetooth.BluetoothAdapter')
BluetoothManager = autoclass('android.bluetooth.BluetoothManager')
PythonActivity = autoclass('org.kivy.android.PythonActivity')


class ServicesView(RecycleView):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.data = []

    def update_services(self, services):
        self.data = [{'text': s} for s in services]


class BLEApp(App):
    def build(self):
        self.activity = PythonActivity.mActivity

        self.adapter = BluetoothManager().getAdapter()

        main_layout = BoxLayout(orientation='vertical', padding=10, spacing=10)

        # Target name input
        input_layout = BoxLayout(orientation='horizontal', size_hint_y=None, height=40)
        input_layout.add_widget(Label(text="Target Device Name:"))
        self.target_input = TextInput(multiline=False)
        input_layout.add_widget(self.target_input)
        main_layout.add_widget(input_layout)

        # Scan button
        self.scan_btn = Button(text="Start Scan", size_hint_y=None, height=40)
        self.scan_btn.bind(on_press=self.start_scan)
        main_layout.add_widget(self.scan_btn)

        # Services list
        self.services_view = ServicesView(size_hint=(1, 1))
        main_layout.add_widget(self.services_view)

        # Status label
        self.status_label = Label(text="Idle", size_hint_y=None, height=40)
        main_layout.add_widget(self.status_label)

        return main_layout

    def start_scan(self, instance):
        target_name = self.target_input.text
        self.status_label.text = "Scanning..."
        self.status_label.color = (1, 1, 0, 1)  # yellow

        services = self.scan_and_connect(target_name)

        if services:
            self.status_label.text = "Connected"
            self.status_label.color = (0, 1, 0, 1)  # green
        else:
            self.status_label.text = "Not Found"
            self.status_label.color = (1, 0, 0, 1)  # red

        self.services_view.update_services(services)

    def scan_and_connect(self, target_name):
        """Scan bonded devices, connect, return list of service UUIDs"""
        bonded = self.adapter.getBondedDevices().toArray()
        target_device = None
        for device in bonded:
            if device.getName() == target_name:
                target_device = device
                break

        if not target_device:
            return []

        # Connect to device and get services (simplified)
        gatt = target_device.connectGatt(self.activity, False, autoclass('android.bluetooth.BluetoothGattCallback')())
        gatt.discoverServices()

        # Wait a bit for services to discover (simplest approach)
        import time
        time.sleep(2)

        services_list = []
        for s in gatt.getServices().toArray():
            services_list.append(s.getUuid().toString())

        return services_list


if __name__ == '__main__':
    BLEApp().run()


JavaException: JVM exception occurred: java.lang.NoClassDefFoundError: android/bluetooth/BluetoothAdapter
java.lang.NoClassDefFoundError: android/bluetooth/BluetoothAdapter
	Caused by:
	java.lang.ClassNotFoundException: android.bluetooth.BluetoothAdapter
	java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:641)
	java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188)
	java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)

In [11]:
!buildozer -v android debug

# Check configuration tokens
Buildozer is running as root!
This is not recommended, and may lead to problems later.
Are you sure you want to continue [y/n]? y
# Ensure build layout
# Create directory /root/.buildozer
# Create directory /root/.buildozer/cache
# Create directory /content/.buildozer
# Create directory /content/bin
# Create directory /content/.buildozer/applibs
# Create directory /root/.buildozer/android/platform/android/platform
# Create directory /content/.buildozer/android/platform
# Create directory /content/.buildozer/android/app
# Check configuration tokens
# Preparing build
# Check requirements for android
# Search for Git (git)
#  -> found at /usr/bin/git
# Search for Cython (cython)
#  -> found at /usr/local/bin/cython
# Search for Java compiler (javac)
#  -> found at /usr/lib/jvm/java-17-openjdk-amd64/bin/javac
# Search for Java keytool (keytool)
#  -> found at /usr/lib/jvm/java-17-openjdk-amd64/bin/keytool
# Install platform
# Run ['git', 'clone', '-b', 'master'

In [12]:
!sudo apt update
!sudo apt install -y \
  openjdk-17-jdk \
  python3-dev \
  git \
  zip \
  unzip \
  cmake \
  autoconf \
  libtool \
  pkg-config \
  zlib1g-dev \
  libncurses5-dev \
  libffi-dev \
  libssl-dev


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,411 kB]

In [13]:
!sudo update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java
!java -version


openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Ubuntu-122.04, mixed mode, sharing)


In [14]:
!pip install --upgrade pip setuptools wheel
!pip install cython
!pip install buildozer python-for-android


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.4/543.4 kB 5.2 MB/s  0:00:00


In [3]:
!buildozer clean android
!buildozer -v android debug

Streaming output truncated to the last 5000 lines.
[DEBUG]:   	./other_builds/hostpython3/desktop/hostpython3/native-build/build/lib.linux-x86_64-3.11/_lzma.cpython-311-x86_64-linux-gnu.so
[DEBUG]:   	./other_builds/hostpython3/desktop/hostpython3/native-build/build/lib.linux-x86_64-3.11/_curses.cpython-311-x86_64-linux-gnu.so
[DEBUG]:   	./other_builds/hostpython3/desktop/hostpython3/native-build/build/lib.linux-x86_64-3.11/audioop.cpython-311-x86_64-linux-gnu.so
[DEBUG]:   	./other_builds/hostpython3/desktop/hostpython3/native-build/build/lib.linux-x86_64-3.11/_curses_panel.cpython-311-x86_64-linux-gnu.so
[DEBUG]:   	./other_builds/hostpython3/desktop/hostpython3/native-build/build/lib.linux-x86_64-3.11/_heapq.cpython-311-x86_64-linux-gnu.so
[DEBUG]:   	./other_builds/hostpython3/desktop/hostpython3/native-build/build/lib.linux-x86_64-3.11/_bisect.cpython-311-x86_64-linux-gnu.so
[DEBUG]:   	./other_builds/hostpython3/desktop/hostpython3/native-build/build/lib.linux-x86_64-3.11/nis.cp

In [7]:
!pip install --upgrade buildozer
!pip install --upgrade python-for-android

In [8]:
!buildozer android clean

# Check configuration tokens
Buildozer is running as root!
This is not recommended, and may lead to problems later.
Are you sure you want to continue [y/n]? y
# Ensure build layout
# Check configuration tokens
# Run ['/usr/bin/python3', '-m', 'pythonforandroid.toolchain', 'clean_builds', '--color=always', '--storage-dir=/content/.buildozer/android/platform/build-arm64-v8a', '--ndk-api=21', '--ignore-setup-py', '--debug']
# Cwd /content/.buildozer/android/platform/python-for-android
[WARNING]: prerequisites.py is experimental and does not support all prerequisites yet.
[WARNING]: Please report any issues to the python-for-android issue tracker.
[WARNING]: prerequisites.py is experimental and does not support all prerequisites yet.
[WARNING]: Please report any issues to the python-for-android issue tracker.
[DEBUG]:   Remove directory and subdirectory /content/.buildozer/android/platform/build-arm64-v8a/build
[DEBUG]:   Create directory /content/.buildozer/android/platform/build-arm64-v8

In [11]:
!buildozer android debug

# Check configuration tokens
Buildozer is running as root!
This is not recommended, and may lead to problems later.
Are you sure you want to continue [y/n]? 
Are you sure you want to continue [y/n]? y
# Ensure build layout
# Check configuration tokens
# Preparing build
# Check requirements for android
# Search for Git (git)
#  -> found at /usr/bin/git
# Search for Cython (cython)
#  -> found at /usr/local/bin/cython
# Search for Java compiler (javac)
#  -> found at /usr/lib/jvm/java-17-openjdk-amd64/bin/javac
# Search for Java keytool (keytool)
#  -> found at /usr/lib/jvm/java-17-openjdk-amd64/bin/keytool
# Install platform
# Run ['git', 'config', '--get', 'remote.origin.url']
# Cwd /content/.buildozer/android/platform/python-for-android
https://github.com/kivy/python-for-android.git
# Run ['git', 'branch', '-vv']
# Cwd /content/.buildozer/android/platform/python-for-android
* master 957a3e5f [origin/master] Merge pull request #2959 from kivy/release-2024.01.21
# Run ['/usr/bin/python3